In [1]:
#Data Process And save
# from numpy.core.fromnumeric import shape
import librosa
import pathlib
import numpy as np
# import pandas as pd
import csv
# import pywt
# from sklearn.decomposition import PCA
# from sklearn import preprocessing
from sklearn.model_selection import train_test_split
# import tensorflow as tf

def get_cwt_mel(path, n_fft, hop_length, n_mels):
    sr = 16000
    duration = 8
    sample_size = sr * duration
    
    y, sr = librosa.load(path, sr=sr, duration=duration)
    file_length = np.size(y)
    
    if file_length < sample_size:
        y = np.concatenate((y, np.zeros(sample_size-file_length)), axis=0)
    else:
        y=y[0:sample_size]

    #mel_spectrogram = librosa.feature.melspectrogram(y, sr, n_fft=2048, hop_length=1001, n_mels=128)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, hop_length=501,n_mfcc=128)
    stft = np.abs(librosa.stft(y,n_fft=254, hop_length=501))
    chroma=librosa.feature.chroma_stft(S=stft,n_chroma=128)
    log_mel_spectrogram = np.concatenate((stft,mfcc,chroma), axis=1)
    log_mel_spectrogram = log_mel_spectrogram.reshape((-1,))
    return log_mel_spectrogram

def get_cwt_mel(path, n_fft, hop_length, n_mels):
    y, sr = librosa.load(path, sr=16000)
    file_length = np.size(y)

    if file_length < 128000:
        y = np.concatenate((y, np.zeros(128000 - file_length)), axis=0)
    else:
        y = y[:128000]

    hop_length = 250 #125=1025, 250=513, 501=256,1001
    mfcc = librosa.feature.mfcc(y=y, sr=sr, hop_length=hop_length, n_mfcc=128)
    stft = np.abs(librosa.stft(y, n_fft=254, hop_length=hop_length))
    chroma = librosa.feature.chroma_stft(S=stft, n_chroma=128)
    log_mel_spectrogram = np.concatenate((mfcc, stft, chroma), axis=1)
    log_mel_spectrogram = log_mel_spectrogram.reshape((-1,))

    # Clear temporary variables
    del mfcc, stft, chroma, y

    return log_mel_spectrogram

def classify_files_new(path):
    dataset_dict = {
        'total': 0,
        'file_dict': {
            'ANGRY': {'represent': 0, 'count': 0, 'all_data': []},
            'HAPPY': {'represent': 1, 'count': 0, 'all_data': []},
            'NEUTRAL': {'represent': 2, 'count': 0, 'all_data': []},
            'SAD': {'represent': 3, 'count': 0, 'all_data': []},
            'SURPRISE': {'represent': 4, 'count': 0, 'all_data': []}
        }
    }

    wav_path = pathlib.Path(path)
    emotion_file_list = [str(file_name) for file_name in wav_path.glob('*.wav')]

    train_data_x, train_data_y = [], []
    test_data_x, test_data_y = [], []
    train_filenames, test_filenames = [], []

    emotion_label_list = dataset_dict['file_dict'].keys()
    for emotion_label in emotion_label_list:
        print(emotion_label)

        emotion_classify_file_list = [letter for letter in emotion_file_list if emotion_label in letter]
        files_count = len(emotion_classify_file_list)
        
        print(files_count)

        dataset_dict['file_dict'][emotion_label]['count'] = files_count
        dataset_dict['total'] += files_count

        emotion_data = [get_cwt_mel(path, n_fft=2048, hop_length=512, n_mels=128) for path in emotion_classify_file_list]

        x = emotion_data
        count = dataset_dict['file_dict'][emotion_label]['count']
        y = np.full(count, dataset_dict['file_dict'][emotion_label]['represent'])
        
        x_train, x_test, y_train, y_test, train_files, test_files = train_test_split(
            x, y, emotion_classify_file_list, train_size=0.80, random_state=1, stratify=y)

        x, y = 128, 513
        z = 3

        train_data_x = np.append(train_data_x, np.array(x_train)).reshape(-1, x, y, z)
        test_data_x = np.append(test_data_x, np.array(x_test)).reshape(-1, x, y, z)

        train_data_y = np.append(train_data_y, y_train)
        test_data_y = np.append(test_data_y, y_test)

        train_filenames.extend(train_files)
        test_filenames.extend(test_files)

    #if the csv folder does not exist, create it
    pathlib.Path(path + '/csv').mkdir(parents=True, exist_ok=True)
    
    np.savetxt(path + '/csv/train_data_y.csv', train_data_y, delimiter=',')
    np.savetxt(path + '/csv/test_data_y.csv', test_data_y, delimiter=',')
    
    with open(path + '/csv/train_filenames.csv', 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['Filename'])
        for filename in train_filenames:
            writer.writerow([filename])

    with open(path + '/csv/test_filenames.csv', 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['Filename'])
        for filename in test_filenames:
            writer.writerow([filename])

    return train_data_x, train_data_y, test_data_x, test_data_y

path = './data/SUBESCOxUIU'
train_data_x, train_data_y, test_data_x, test_data_y = classify_files_new(path)

ANGRY
900
HAPPY
900
NEUTRAL
900
SAD
900
SURPRISE
900


In [1]:
path = './data/SUBESCOxUIU'

In [2]:
import numpy as np
def save_array_to_npy(array, file_name):
    np.save(file_name, array)

def load_array_from_npy(file_name):
    return np.load(file_name)

#if saved directory does not exist, create it
pathlib.Path(path + '/saved').mkdir(parents=True, exist_ok=True)

# Example usage
save_array_to_npy(train_data_x, path + '/saved/train_data_x.npy')
save_array_to_npy(train_data_y, path + '/saved/train_data_y.npy')
save_array_to_npy(test_data_x, path + '/saved/test_data_x.npy')
save_array_to_npy(test_data_y, path + '/saved/test_data_y.npy')

del train_data_x, train_data_y, test_data_x, test_data_y
# Loading the arrays
train_data_x = load_array_from_npy(path + '/saved/train_data_x.npy')
train_data_y = load_array_from_npy(path + '/saved/train_data_y.npy')
test_data_x = load_array_from_npy(path + '/saved/test_data_x.npy')
test_data_y = load_array_from_npy(path + '/saved/test_data_y.npy')

print('train_data_x shape:', train_data_x.shape)
print('train_data_y shape:', train_data_y.shape)
print('test_data_x shape:', test_data_x.shape)
print('test_data_y shape:', test_data_y.shape)

train_data_x shape: (3600, 128, 513, 3)
train_data_y shape: (3600,)
test_data_x shape: (900, 128, 513, 3)
test_data_y shape: (900,)


In [1]:
from numpy.core.fromnumeric import shape
import librosa
import pathlib
import numpy as np
import os
# import pywt
import csv
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.utils import normalize, to_categorical
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

import tensorflow as tf
from tensorflow import keras,nn
from tensorflow.keras import layers

def model3d(input_shape, num_classes):

    model = keras.Sequential(name='model3d')

    #LFLB1
    model.add(layers.Conv3D(filters=64,kernel_size=3,strides=1,padding='same',input_shape=input_shape))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling3D(pool_size=(2,2,1), strides=(2,2,1)))
    
    #LFLB2
    model.add(layers.Conv3D(filters=64,kernel_size=3,strides=1, padding='same', ))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling3D(pool_size=(4,4,1), strides=(4,4,1)))
    #model.add(layers.MaxPooling2D(pool_size=2, strides=2))

    #LFLB3
    model.add(layers.Conv3D(filters=128,kernel_size=3,strides=1,padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling3D(pool_size=(4,4,1), strides=(4,4,1)))
    #model.add(layers.MaxPooling2D(pool_size=2, strides=2))

    
    #LFLB4
    model.add(layers.Conv3D(filters=128,kernel_size=3,strides=1,padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling3D(pool_size=(4,4,1), strides=(4,4,1)))

    #model.add(layers.Reshape((-1, 128)))
    model.add(layers.TimeDistributed(layers.Flatten()))
    
    #LSTM
    #model.add(layers.LSTM(256))
    model.add(layers.Bidirectional(layers.LSTM(256)))
    
    #model.add(keras.layers.Dense(128,activation=nn.relu))
    #model.add(layers.Dense(128,activation=nn.relu))
    model.add(layers.Dense(units=num_classes, activation='softmax'))

    model.summary()

    opt = keras.optimizers.Adam(learning_rate=0.0006, decay=1e-6)

    model.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['categorical_accuracy'])

    return model

def model2dv2(input_shape, num_classes):

    model = keras.Sequential(name='model2d')

    #LFLB1
    model.add(layers.Conv2D(filters=64,kernel_size=(3,3),strides=1,padding='same',input_shape=input_shape))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling2D(pool_size=2, strides=2))
    
    #LFLB2
    model.add(layers.Conv2D(filters=64,kernel_size=(3,3),strides=1, padding='same', ))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling2D(pool_size=(4,4), strides=(4,4)))
    #model.add(layers.MaxPooling2D(pool_size=2, strides=2))

    #LFLB3
    model.add(layers.Conv2D(filters=128,kernel_size=(3,3),strides=1,padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling2D(pool_size=(4,4), strides=(4,4)))
    #model.add(layers.MaxPooling2D(pool_size=2, strides=2))


    
    #LFLB4
    model.add(layers.Conv2D(filters=128,kernel_size=(3,3),strides=1,padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling2D(pool_size=(4,4), strides=(4,4)))

    #model.add(layers.Reshape((-1, 128)))
    model.add(layers.TimeDistributed(layers.Flatten()))
    
    #LSTM
    #model.add(layers.LSTM(256))
    model.add(layers.Bidirectional(layers.LSTM(256)))
    
    #model.add(keras.layers.Dense(128,activation=nn.relu))
    #model.add(layers.Dense(128,activation=nn.relu))
    model.add(layers.Dense(units=num_classes, activation='softmax'))

    model.summary()

    opt = keras.optimizers.Adam(learning_rate=0.0006, decay=1e-6)

    model.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['categorical_accuracy'])

    return model

def model2d(input_shape, num_classes):

    model = keras.Sequential(name='model2d')

    #LFLB1
    model.add(layers.Conv2D(filters=64,kernel_size=3,strides=1,padding='same',input_shape=input_shape))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling2D(pool_size=2, strides=2))
    
    #LFLB2
    model.add(layers.Conv2D(filters=64,kernel_size=3,strides=1, padding='same', ))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling2D(pool_size=4, strides=4))
    #model.add(layers.MaxPooling2D(pool_size=2, strides=2))

    #LFLB3
    model.add(layers.Conv2D(filters=128,kernel_size=3,strides=1,padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling2D(pool_size=4, strides=4))
    #model.add(layers.MaxPooling2D(pool_size=2, strides=2))


    
    #LFLB4
    model.add(layers.Conv2D(filters=128,kernel_size=3,strides=1,padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling2D(pool_size=4, strides=4))

    #model.add(layers.Reshape((-1, 128)))
    model.add(layers.TimeDistributed(layers.Flatten()))
    
    #LSTM
    #model.add(layers.LSTM(256))
    model.add(layers.Bidirectional(layers.LSTM(256)))
    
    #model.add(keras.layers.Dense(128,activation=nn.relu))
    #model.add(layers.Dense(128,activation=nn.relu))
    model.add(layers.Dense(units=num_classes, activation='softmax'))

    model.summary()

    opt = keras.optimizers.Adam(learning_rate=0.0006, decay=1e-6)

    model.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['categorical_accuracy'])

    return model

def model2CNN2F(input_shape, num_classes):

    model = keras.Sequential(name='model2d')

    #LFLB1
    model.add(layers.Conv2D(filters=128,kernel_size=3,strides=1,padding='same',input_shape=input_shape))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('elu'))
    model.add(layers.MaxPooling2D(pool_size=4, strides=4))

    #LFLB3
    model.add(layers.Conv2D(filters=64,kernel_size=3,strides=1,padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('elu'))
    model.add(layers.MaxPooling2D(pool_size=8, strides=8))

    #model.add(layers.Reshape((-1, 128)))
    model.add(layers.Flatten())
    model.add(layers.Dense(128,activation=nn.relu))
    model.add(layers.Dense(units=num_classes, activation='softmax'))

    model.summary()

    opt = keras.optimizers.Adam(learning_rate=0.0006, decay=1e-6)

    model.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['categorical_accuracy'])

    return model


# Clear TensorFlow session to release resources
tf.keras.backend.clear_session()

# Ensure memory growth is set
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
else:
    print("No GPU found. Training will proceed on the CPU.")
    
def save_array_to_npy(array, file_name):
    np.save(file_name, array)

def load_array_from_npy(file_name):
    return np.load(file_name)

# path = './data/SUBESCO_200'

def train(train_data_x, train_data_y, emotion, emotionNumber, epochs=10):
    model = model2d(input_shape=(128, 513, 3), num_classes=emotionNumber)
    model.summary()
    
    batch_size = 5;
    
    
    es = EarlyStopping(monitor='val_categorical_accuracy',mode='max',verbose=1,patience=20)
    
    mc = ModelCheckpoint(path+'/model/'+emotion+'_max_model.keras',monitor='val_categorical_accuracy',mode='max',verbose=1,save_best_only=True)
    
    history=model.fit(train_data_x, train_data_y, validation_data=(test_data_x, test_data_y) ,epochs=epochs, batch_size=batch_size, verbose=2, callbacks=[es,mc])
    
    acc=history.history['categorical_accuracy'][len(history.history['categorical_accuracy']) - 1]
    
    model.save(path+'/model/'+emotion+'_max_model.keras')
    
    return acc

def test(test_data_x, test_data_y, emotion):
    
    new_model = load_model(path+'/model/'+emotion+'_max_model.keras')
    history=new_model.evaluate(test_data_x, test_data_y, batch_size=1)
    predict=new_model.predict(test_data_x)
    return history[1]

def maxIndex(data):
  max=data[0]
  index=0
  for i in range(1,len(data)):
    if(max<data[i]):
      max=data[i]
      index=i
  return index  

def test_emotion(test_data_x, test_data_y, total, emotion, path):
    model_path = os.path.join(path, 'model', emotion + '_max_model.keras')
    
    # Check if the file exists before loading
    if not os.path.isfile(model_path):
        raise ValueError(f"File not found: {model_path}. Please ensure the file is an accessible `.keras` zip file.")
    
    new_model = load_model(model_path)
    
    # Ensure the labels are one-hot encoded for testing
    test_data_y = to_categorical(test_data_y, num_classes=total)
    
    # Evaluate the model
    history = new_model.evaluate(test_data_x, test_data_y, batch_size=5)
    print(f"Test accuracy: {history[1]}")
    
    # Predict and generate confusion matrix
    predictions = new_model.predict(test_data_x, batch_size=5)
    confusion_matrix = np.zeros((total, total))
    
    for i in range(len(test_data_y)):
        actual_label = np.argmax(test_data_y[i])
        predicted_label = np.argmax(predictions[i])
        confusion_matrix[actual_label][predicted_label] += 1
    
    # Ensure the directory exists
    save_dir = os.path.join(path, 'data')
    os.makedirs(save_dir, exist_ok=True)
    
    # Save the confusion matrix to CSV
    confusion_csv_path = os.path.join(save_dir, emotion + '_confusion.csv')
    with open(confusion_csv_path, "w", newline='') as my_csv:
        csvWriter = csv.writer(my_csv, delimiter=',')
        csvWriter.writerows(confusion_matrix)
    
    print(f"Confusion matrix saved to: {confusion_csv_path}")


d:\AI\BSER\.conda\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


In [4]:
train_data_y = to_categorical(train_data_y)
test_data_y = to_categorical(test_data_y)
emotion='SUBESCOxUIU'

acc=train(train_data_x, train_data_y, emotion, 5, 50)


Model: "model2d"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 128, 513, 64)      1792      
                                                                 
 batch_normalization (BatchN  (None, 128, 513, 64)     256       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 128, 513, 64)      0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 64, 256, 64)      0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 256, 64)       36928     
                                                                 
 batch_normalization_1 (Batc  (None, 64, 256, 64)      256 

In [3]:
def test_emotion(test_data_x, test_data_y, total, emotion, path):
    model_path = os.path.join(path, 'model', 'SUBESCOxUIU_max_model.keras')
    
    print(model_path)
    
    # Check if the file exists before loading
    if not os.path.isfile(model_path):
        raise ValueError(f"File not found: {model_path}. Please ensure the file is an accessible `.keras` zip file.")
    
    new_model = load_model(model_path)
    
    # Ensure the labels are one-hot encoded for testing
    test_data_y = to_categorical(test_data_y, num_classes=total)
    
    # Evaluate the model
    history = new_model.evaluate(test_data_x, test_data_y, batch_size=5)
    print(f"Test accuracy: {history[1]}")
    
    # Predict and generate confusion matrix
    predictions = new_model.predict(test_data_x, batch_size=5)
    confusion_matrix = np.zeros((total, total))
    
    for i in range(len(test_data_y)):
        actual_label = np.argmax(test_data_y[i])
        predicted_label = np.argmax(predictions[i])
        confusion_matrix[actual_label][predicted_label] += 1
    
    # Ensure the directory exists
    save_dir = os.path.join(path, 'data')
    os.makedirs(save_dir, exist_ok=True)
    
    # Save the confusion matrix to CSV
    confusion_csv_path = os.path.join(save_dir, emotion + '_confusion.csv')
    with open(confusion_csv_path, "w", newline='') as my_csv:
        csvWriter = csv.writer(my_csv, delimiter=',')
        csvWriter.writerows(confusion_matrix)
    
    print(f"Confusion matrix saved to: {confusion_csv_path}")
    

path = './data/SUBESCOxUIU'

# test_data_x = load_array_from_npy(path + '/saved/test_data_x.npy')
# test_data_y = load_array_from_npy(path + '/saved/test_data_y.npy')

test_data_x = load_array_from_npy('./data/test-data/UIU/subset_81/saved/test_data_x.npy')
test_data_y = load_array_from_npy('./data/test-data/UIU/subset_81/saved/test_data_y.npy')

print(test_data_x.shape)
print(test_data_y.shape)

test_emotion(test_data_x, test_data_y, 5, 'TR-comb-TST-uiu', path)


(400, 128, 513, 3)
(400,)
./data/SUBESCOxUIU\model\SUBESCOxUIU_max_model.keras
80/80 [==============================] - 3s 14ms/step - loss: 1.2198 - categorical_accuracy: 0.7475
Test accuracy: 0.7475000023841858
80/80 [==============================] - 1s 10ms/step
Confusion matrix saved to: ./data/SUBESCOxUIU\data\TR-comb-TST-uiu_confusion.csv


In [6]:
print(path)

./data/UIU
